In [1]:
import pandas as pd
import spacy, re
import numpy as np
from collections import Counter, defaultdict
from unidecode import unidecode
import seaborn as sns
from lemmatization import ohe_df

# One Hot Encoding

In [ ]:
mails = pd.read_csv('data/mails_lemmatized.csv', index_col = 0, encoding='UTF-16', header=0, skipinitialspace=True, skip_blank_lines = True)
print(mails.shape)
mails.head()

(3407, 6)


,body,header,date,from,to,label
0,yohan quand tu as le temps dis moi si c est cl...,1ère relecture gt conso,"Fri, 29 May 2020 16:53:04 +0200",plonquet nadège,NaN,0
1,pour accepter la demande cliquez simplement su...,accepter,"Mon, 16 Sep 2019 17:09:37 +0200",plonquet nadège,NaN,0
2,bonjour pouvez vous donner les accès à decibel...,acces decibel,"Tue, 23 Mar 2021 13:44:07 +0100",guillaume veronique,NaN,1
3,comme ça ne fonctionne toujours pas sur mon po...,accès rec4 decibel sdw rec4 hm dm ad restituti...,"Mon, 24 Feb 2020 14:28:57 +0100",gueniot bernard,NaN,1
4,tu sais ce que c est que ces actes qui n ont p...,actes indemnités hospitalières,"Mon, 25 Mar 2019 11:25:36 +0100",levisse xavier,NaN,1


In [6]:
data = ohe_df(mails)
data

,1ere,relecture,gt,consommation,yohan,temps,dire,clair,mettre,decibel,...,bulletin,digiposte,voltaire,alaoui,enora,samber,lanrivain,batard,leudet,demande_de_support
0,1,1,1,1,1,1,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3402,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3403,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3404,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3405,0,0,0,0,0,1,1,0,2,0,...,3,1,1,0,0,0,0,0,0,0


In [7]:
Counter(data['demande_de_support'])

Counter({0: 2755, 1: 455, 2: 197})

In [9]:
data.to_csv('data/mails_ohe.csv')

## Apprentissage semi-supervisé

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
plt.style.use('ggplot')

In [10]:
X_unlabeled = data.loc[data["demande_de_support"] == 2].drop(columns=['demande_de_support'])
X = data.loc[data["demande_de_support"] != 2].drop(columns=['demande_de_support'])
y = pd.Series(data.loc[data["demande_de_support"] != 2]['demande_de_support'], dtype=int)

In [8]:
from sklearn.cluster import KMeans

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
kmeans1 = KMeans(n_clusters=2)
kmeans1.fit(X_train, y_train)

y_pred = kmeans1.predict(X_test)
accuracy_score(y_test,y_pred)

0.7870074547390842

In [9]:
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model_svc = SVC(C=0.1325, kernel = 'linear')
model_svc.fit(X_train, y_train)

y_pred = model_svc.predict(X_test)
accuracy_score(y_test, y_pred)

0.9126730564430245

In [11]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
LR = LogisticRegression(C=1, solver = 'newton-cg')
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy_score(y_test, y_pred)

0.9137380191693291

### Prédiction des labels manquants

In [13]:
#estimator = SVC(C=0.1325, kernel = 'linear', probability = True)
estimator = LogisticRegression(C=1, solver = 'newton-cg')
X_unlabeled_copy = X_unlabeled.copy()
newX = X
newY = y
threshold = 0.9
print(newX.shape, X_unlabeled_copy.shape)
while not X_unlabeled_copy.empty:

    estimator.fit(newX, newY)
    y_pred = estimator.predict(X_unlabeled_copy)
    y_prob = estimator.predict_proba(X_unlabeled_copy)
    X_unlabeled_copy['_pred_'] = y_pred
    X_unlabeled_copy['_prob_'] = [y_prob[k,y_pred[k]] for k in range(len(y_pred))]
    confident_pred = X_unlabeled_copy[X_unlabeled_copy['_prob_'] >= threshold]
    X_unlabeled_copy = X_unlabeled_copy[X_unlabeled_copy['_prob_'] < threshold].drop(['_pred_', '_prob_'], axis = 1)
    newX = pd.concat([newX, confident_pred.drop(['_pred_', '_prob_'], axis = 1)], ignore_index = True)
    newY = pd.concat([newY, confident_pred['_pred_']], ignore_index = True)
    print(newX.shape, X_unlabeled_copy.shape, threshold)

    if len(confident_pred)==0:
        break

(3127, 10359) (169, 10359)
(3271, 10359) (25, 10359) 0.8
(3274, 10359) (22, 10359) 0.8
(3274, 10359) (22, 10359) 0.8


# Catégorisation des mails

In [96]:
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)
kmeans1 = KMeans(n_clusters=2)
kmeans1.fit(X_train, y_train)

y_pred = kmeans1.predict(X_test)
accuracy_score(y_test,y_pred)

0.7367886178861789

In [97]:
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)
model_svc = SVC(C=0.1325, kernel = 'linear')
model_svc.fit(X_train, y_train)

y_pred = model_svc.predict(X_test)
accuracy_score(y_test, y_pred)

0.9054878048780488

In [101]:
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)
LR = LogisticRegression(C=1, solver = 'newton-cg')
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy_score(y_test, y_pred)

0.9105691056910569

## Optimisation des modèles

In [48]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)

gridCV = GridSearchCV(SVC(),
    n_jobs=-1,
    param_grid = {
        'C' : [0.1325],
        'kernel' : ['linear'],
        #'degree' : [1, 2, 3, 4],
        #'gamma' : ['scale', 'auto'] + list(np.linspace(0,2,3)),
        #'coef0' : np.linspace(-2,2,5),
        'class_weight' : ['balanced', None],
        #'decision_function_shape' : ['ovo', 'ovr']
    }
)

gridCV.fit(X_train, y_train)
print(gridCV.best_params_)
print(gridCV.best_score_)
#{'C': 0.1325, 'class_weight': None, 'kernel': 'linear'}

{'C': 0.1325, 'class_weight': None, 'kernel': 'linear'}
0.9192010235179116


In [105]:
model_svc = SVC(C=0.1325, kernel = 'linear')
model_svc.fit(X_train, y_train)
y_pred = model_svc.predict(X_test)
accuracy_score(y_test, y_pred)

0.9105691056910569

In [ ]:
filename = 'svc_c01325_lin.sav'
with open(filename, 'wb') as file:
    pickle.dump(model_svc, file)

In [ ]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)

gridCV2 = GridSearchCV(LogisticRegression(),
    n_jobs=-1,
    param_grid = {
'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
'dual' : [False, True],
'C' : np.linspace(0.0001,3, 10),
'fit_intercept' : [True, False],
'class_weight' : [None, 'balanced'],
'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
'warm_start' : [True],
'l1_ratio' : [0, 0.5, 1]
    }
)

gridCV2.fit(X_train, y_train)
print(gridCV2.best_params_)
print(gridCV2.best_score_)
#{'C': 0.3334222222222222, 'class_weight': None, 'dual': False, 'fit_intercept': False, 'l1_ratio': 0, 'penalty': 'l1', 'solver': 'liblinear', 'warm_start': True}
#0.91

In [24]:
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.3)
LR = LogisticRegression(C = 0.3334222222222222, class_weight = None, dual = True, fit_intercept = False, 
                        solver = 'liblinear', warm_start = True)
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\le_paumier-m\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.9196337741607324

In [116]:
filename = 'lr_c033_liblinear_dual.sav'
with open(filename, 'wb') as file:
    pickle.dump(LR, file)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
LR = LogisticRegression(C = 0.3334222222222222, class_weight = None, dual = True, fit_intercept = False, 
                        solver = 'liblinear', warm_start = True)
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\le_paumier-m\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.9116080937167199

Sur ces méthodes de machine learning classique, l'extension du dataset à partir des données non-labellisées ne produit pas une grande amélioration des performances, ce sera plus flagrant lorsqu'on passera à des méthodes de Deep Learning

## Prédiction des probabilités

In [117]:
np.round(LR.predict_proba(newX),2)

array([[0.99, 0.01],
       [0.78, 0.22],
       [0.14, 0.86],
       ...,
       [0.94, 0.06],
       [0.98, 0.02],
       [0.66, 0.34]])

## Exemple de pipeline

In [15]:
from onehotencoder import countLemmas, initSpacy
lemmatizer = initSpacy('fr_core_news_md')

In [33]:
input_mail = """
    DRS - 
    Bonjour
    j'ai un souci avec le décisionnel DECIBEL, il y a un problème de référenciel.

    Cordialement,
    Martin
    """

In [29]:
def pipeline(mail, model_file, schema, lemmatizer):
    features = defaultdict(int)
    features.update(countLemmas(mail, lemmatizer))
    X = np.array([features[key] for key in schema]).reshape(1, -1)    
    with open(model_file, 'rb') as file:
        model = pickle.load(file)

    try:
        probas = model.predict_proba(X)[0]
    except AttributeError:
        probas = [None, None]
    
    y = model.predict(X)[0]
    return {'predicted_class' : y, 'confidence' : probas[y], 'model' : model_file}

In [34]:
model_file = 'lr_c1_newtoncg.sav'
schema = X.columns.to_list()
predicted_class = pipeline(input_mail, model_file, schema, lemmatizer)
predicted_class

array([1])

to-do :

* vote majoritaire entre plusieurs modèles
* refit les modèles en infériorité numérique / refit périodiquement les moins bons modèles (p.r à un score de prédiction calculé sur la période)
* gerer les cas douteux grace à la proba
* LSTM / biLSTM
* Transformers